# Experiment Proper

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

# Add more CTM models here as needed
from ctmmodels.ringbarrier import DTSimplexRingBarrier as NewModel
from ctmmodels.parentmodel import ParentModel as OldModel

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'dataframes/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

In [4]:
def run_model(demand, weights=(0.4, 0.4, 0.2), new_model=True, log_output=True):
    if new_model:
        model = NewModel(
            demand=demand,
            alpha=weights[0],
            beta=weights[1],
            gamma=weights[2],
            **parameters
        )
    else:
        model = OldModel(
            demand=demand,
            **parameters
        )

    model.generate()
    runtime = model.solve(log_output=log_output)
    dfx, dfy, dfg = model.return_solution()
    dfparams = model.return_parameters()
    obj_values = model.return_objective_value()
    return dfx, dfy, dfg, dfparams, obj_values, runtime, model

In [5]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Steps for Experiment Running

1. Generate the initial greentimes and objective values for perusal later in the discussion
2. Use the initial greentimes in running the LocalSim simulator (USE THE SAME PARAMETERS)
3. Record the animation
4. Obtain the results and preprocess for later

## Current Parameters

* **6-20 seconds** greentime range
* **30 time steps** or 60 seconds
* **5 runs** in LocalSim

## 1. Generate initial greentimes

In [7]:
simplex_points = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

In [8]:
# Generate the new model greentimes

_df_tuples = []
vols = []
delqs = []

for d in demands:
    for pt in simplex_points:
        dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, weights=pt, new_model=True, log_output=True)
        
        # Save the dataframes
        save_df(dfx, 'final_test/volumes/volumes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfy, 'final_test/flows/flows_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfg, 'final_test/greentimes/greentimes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        
        # Get additional results
        vol = model.return_volume()
        delq = model.return_delay_equity()
        
        # Save the other results
        if isinstance(d, int):
            d_tuple = (d,d)
        else:
            d_tuple = d

        _df_tuples.append(
            (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], pt[0], pt[1], pt[2])
        )
        
        vols.append(
            (d_tuple[0], d_tuple[1], vol[0], vol[1], vol[2], vol[3], pt[0], pt[1], pt[2])
        )
        
        delqs.append(
            (d_tuple[0], d_tuple[1], delq[0], delq[1], delq[2], delq[3], pt[0], pt[1], pt[2])
        )
    
    # Old model
    dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, new_model=False, log_output=True)
        
    # Save the dataframes
    save_df(dfx, 'final_test/volumes/volumes_d{}_old'.format(d))
    save_df(dfy, 'final_test/flows/flows_d{}_old'.format(d))
    save_df(dfg, 'final_test/greentimes/greentimes_d{}_old'.format(d))
        
    # Get additional results
    vol = model.return_volume()
    delq = model.return_delay_equity()

    # Save the other results
    if isinstance(d, int):
        d_tuple = (d,d)
    else:
        d_tuple = d

    _df_tuples.append(
        (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], 0, 0, 0)
    )
        
    vols.append(
        (d_tuple[0], d_tuple[1], vol[0], vol[1], vol[2], vol[3], 0, 0, 0)
    )

    delqs.append(
        (d_tuple[0], d_tuple[1], delq[0], delq[1], delq[2], delq[3], 0, 0, 0)
    )

Solving...
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3188 rows and 292 columns.
MIP Presolve modified 756 coefficients.
Aggregator did 158 substitutions.
Reduced MIP has 3456 rows, 1890 columns, and 11432 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (13.14 ticks)
Found incumbent of value 1720.044950 after 0.05 sec. (24.11 ticks)
Probing fixed 0 vars, tightened 104 bounds.
Probing time = 0.01 sec. (0.55 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 600 rows and 104 columns.
Aggregator did 84 substitutions.
Reduced MIP has 2772 rows, 1702 columns, and 9940 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (8.07 ticks)
Probing time = 0.00 sec. (1.19 ticks)
Clique table members: 540.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8

  71073 23798        cutoff            714.2682      683.9280  3520641    4.25%
  72701 23922      709.8637    97      714.2682      684.5438  3580175    4.16%
  74165 24141        cutoff            714.2682      685.3190  3676602    4.05%
  75775 24235        cutoff            714.2682      686.0470  3727511    3.95%
  77423 24353        cutoff            714.2682      686.6754  3790421    3.86%
  79071 24394      694.5688   120      714.2682      687.2653  3865299    3.78%
  80608 24517      711.9685    99      714.2682      687.7971  3959767    3.71%
* 82147+24539                          709.5390      688.4374             2.97%
  82228 24534      702.3247    93      709.5390      688.4374  4018363    2.97%
Elapsed time = 125.27 sec. (63990.32 ticks, tree = 22.71 MB, solutions = 12)
  83486 20704        cutoff            709.5390      689.1750  4102115    2.87%
  85040 20496        cutoff            709.5390      689.7794  4149680    2.78%
  86785 20223        cutoff            709.

  19030  9783        cutoff           -200.1184     -222.7578  1373521   11.31%
  19942 10251     -212.4379   130     -200.1184     -222.4697  1423408   11.17%
  20882 11014     -211.0092   104     -200.1184     -222.1550  1490363   11.01%
  21802 11422        cutoff           -200.1184     -221.9159  1531320   10.89%
  22755 11860     -212.6508   124     -200.1184     -221.6293  1577301   10.75%
Elapsed time = 61.22 sec. (35462.95 ticks, tree = 185.32 MB, solutions = 6)
  23659 12516     -204.0696   101     -200.1184     -221.3416  1633633   10.61%
  24617 12855     -207.1450    46     -200.1184     -221.1164  1670829   10.49%
  25622 13464     -212.8689    91     -200.1184     -220.8893  1725599   10.38%
  26475 14144     -212.0184   127     -200.1184     -220.7038  1793068   10.29%
  27469 14559        cutoff           -200.1184     -220.4888  1844539   10.18%
  28432 15019     -202.4792    55     -200.1184     -220.3376  1884371   10.10%
* 28694+15350                         -200.2

Found incumbent of value 0.000000 after 0.03 sec. (24.10 ticks)
Probing fixed 0 vars, tightened 104 bounds.
Probing time = 0.00 sec. (0.55 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 600 rows and 104 columns.
Aggregator did 84 substitutions.
Reduced MIP has 2772 rows, 1702 columns, and 9940 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (8.11 ticks)
Probing time = 0.00 sec. (1.19 ticks)
Clique table members: 540.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.18 sec. (178.32 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000    -8625.0044              --- 
      0     0    -2099.7890   234        0.0000    -2099.7890     2465     --- 
*     0+    0    

  49711 25245    -2058.6366    87    -2055.8017    -2074.3652  3318022    0.90%
  50635 25936        cutoff          -2055.8017    -2074.1918  3381399    0.89%
  51506 26315    -2069.1532   102    -2055.8017    -2074.0600  3422724    0.89%
  52454 26484    -2062.0278   102    -2055.8017    -2074.0040  3441734    0.89%
  53413 27193        cutoff          -2055.8017    -2073.8699  3531424    0.88%
  54465 27452    -2057.3728   118    -2055.8017    -2073.7281  3554102    0.87%
  55393 27948    -2059.9109   137    -2055.8017    -2073.6038  3601061    0.87%
  56230 28498    -2060.3806    93    -2055.8017    -2073.4259  3667485    0.86%
  57190 28842        cutoff          -2055.8017    -2073.3256  3717113    0.85%
Elapsed time = 133.70 sec. (74627.52 ticks, tree = 435.68 MB, solutions = 7)
  58245 28976    -2067.5714    99    -2055.8017    -2073.1967  3737323    0.85%
  59317 29647    -2060.1247   122    -2055.8017    -2073.1232  3804056    0.84%
  60260 29808    -2066.9300   104    -2055.

...

  25615  5904      100.6510    48       99.8196      101.9383  1600561    2.12%
  26538  5821       99.8822    64       99.8196      101.8196  1647739    2.00%
  27435  5715        cutoff             99.8196      101.6707  1699862    1.85%
  28430  5479        cutoff             99.8196      101.5450  1755708    1.73%
  29387  5333        cutoff             99.8196      101.3897  1788679    1.57%
Elapsed time = 79.17 sec. (46080.59 ticks, tree = 81.12 MB, solutions = 13)
* 30338  5048      integral     0      100.1392      101.2707  1839922    1.13%
  30377  5087        cutoff            100.1392      101.2707  1831624    1.13%
  31500  3427       98.7904    89      100.1392      101.0242  1896662    0.88%
  32752  2543      100.1798   100      100.1392      100.8212  1942942    0.68%
  34556  1168        cutoff            100.1392      100.4589  1987224    0.32%

Cover cuts applied:  78
Implied bound cuts applied:  9
Flow cuts applied:  8
Mixed integer rounding cuts applied:  8
Lift a

   1794   805     -246.7265    68     -241.4773     -250.0000   185999    3.53%
   2161  1071        cutoff           -241.4773     -250.0000   233930    3.53%
   2556  1253     -246.4313    87     -241.4773     -250.0000   265269    3.53%
   3233  1570     -244.2464   101     -241.4773     -249.1775   323455    3.19%
   4205  1872     -242.6847    28     -241.4773     -248.6005   366262    2.95%
   4986  2247     -250.0000   187     -241.4773     -248.0117   426886    2.71%
   5002   826     -250.0000   142     -241.4773     -248.0117   432484    2.71%
   5263   124     -248.0883   132     -241.4773     -248.0117   482380    2.71%
   5618   325     -247.0509    81     -241.4773     -248.0117   549854    2.71%
   6031   495     -248.0366    88     -241.4773     -248.0117   597072    2.71%
Elapsed time = 24.56 sec. (15114.36 ticks, tree = 3.41 MB, solutions = 7)
   6544   813     -247.1922    50     -241.4773     -248.0117   643304    2.71%
   7222  1100        cutoff           -241.477

  12396  2362    -3672.8244    87    -3670.5696    -3675.3165  1169984    0.13%
  12893  2589    -3673.3085    78    -3670.5696    -3675.3165  1215252    0.13%
  13503  2747        cutoff          -3670.5696    -3675.3165  1252264    0.13%
  13890  2903    -3670.6841   118    -3670.5696    -3675.3165  1284766    0.13%
  14297  3013    -3674.7473    84    -3670.5696    -3675.3165  1313418    0.13%
Elapsed time = 56.09 sec. (34172.04 ticks, tree = 60.40 MB, solutions = 6)
  14760  3120    -3670.6224    52    -3670.5696    -3675.3165  1331331    0.13%
  15258  3286    -3674.3012   111    -3670.5696    -3675.3165  1380500    0.13%
  15907  3376        cutoff          -3670.5696    -3675.3165  1417227    0.13%
  16656  3465    -3671.9550   108    -3670.5696    -3675.0737  1454533    0.12%
  17764  3564        cutoff          -3670.5696    -3674.4678  1502714    0.11%
  18696  3640    -3673.1013    64    -3670.5696    -3674.1561  1539330    0.10%
  19461  3635    -3673.5584    79    -3670.56

    591   286    -1962.6582   161    -1953.8928    -1962.6582    46053    0.45%
    721   382    -1962.4830    92    -1953.8928    -1962.6582    59492    0.45%
    881   442        cutoff          -1953.8928    -1962.6582    68709    0.45%
   1013   549        cutoff          -1953.8928    -1962.6582    82292    0.45%
   1089   590    -1959.0270   132    -1953.8928    -1962.6582    89591    0.45%
   1650   898    -1953.9883    62    -1953.8928    -1962.6582   136231    0.45%
Elapsed time = 7.25 sec. (4450.10 ticks, tree = 18.64 MB, solutions = 7)
*  2052+ 1180                        -1954.9581    -1962.6582             0.39%
   2126  1098    -1958.1224   120    -1954.9581    -1962.6582   188986    0.39%
   2823  1415        cutoff          -1954.9581    -1962.4955   236781    0.39%
   3336  1738    -1962.6582   143    -1954.9581    -1962.1374   289567    0.37%
   3376    46    -1956.0116   130    -1954.9581    -1962.1374   295086    0.37%
   3896   301    -1957.3455    67    -1954.9581

*     0+    0                         1665.8719    -3627.3071           317.74%
      0     0     -209.6618   137     1665.8719     -209.6618        0  112.59%
*     0+    0                         -110.6888     -209.6618            89.42%
      0     0     -206.0837   154     -110.6888      Cuts: 21      158   86.18%
      0     0     -197.1723   167     -110.6888     Cuts: 119      411   78.13%
      0     0     -197.1129   167     -110.6888      Cuts: 79      434   78.08%
*     0+    0                         -127.7447     -197.1129            54.30%
      0     0     -197.0541   171     -127.7447      Cuts: 10      457   54.26%
      0     0     -197.0537   172     -127.7447       Cuts: 3      461   54.26%
*     0+    0                         -132.4268     -197.0537            48.80%
      0     0     -197.0537   168     -132.4268    MIRcuts: 4      463   48.80%
*     0+    0                         -150.4691     -197.0537            30.96%
*     0+    0                         -1

In [9]:
df = pd.DataFrame(data=_df_tuples,columns=['demand_ns', 'demand_ew', 'runtime', 'delay', 'throughput', 'objective_value', 'alpha', 'beta', 'gamma'])
save_df(df, "final_test/results")

In [10]:
df_vols = pd.DataFrame(data=vols,columns=['demand_ns', 'demand_ew', 'volume_n', 'volume_e', 'volume_s', 'volume_w', 'alpha', 'beta', 'gamma'])
save_df(df_vols, "final_test/volume_equity")

In [11]:
df_delqs = pd.DataFrame(data=delqs,columns=['demand_ns', 'demand_ew', 'delay_n', 'delay_e', 'delay_s', 'delay_w', 'alpha', 'beta', 'gamma'])
save_df(df_delqs, "final_test/delay_equity")

## 2. Run the experiments

**Notes:**

* USE THE SAME PARAMETERS
* Label the results Excel sheets later